In [ ]:
!pip install tensorflow pandas matplotlib sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install tensorflow


In [ ]:
!pip install -v tensorflow-gpu


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Running command python setup.py egg_info
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/dist-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
      parsed = parse_requirement(requirement_string)
    File "/usr/local/lib/python3.10/dist-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
    File "/usr/local/lib/python3.10/dist-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
    File "/usr/local/lib/python3.10/dist-packages/setuptools/_vendor/packaging/_parser.py", line 126, in _parse_requirement_details
      marker = _parse_requirement_marker(
    File "/usr/local/lib/python3.10/dist-packages/setuptools/_vendor/packaging/_parser.py", line 147, in _par

In [ ]:
!pip search tensorflow-gpu


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [ ]:
!pip install tensorflow-gpu==<version>


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install tensorflow-gpu==<version>'


In [ ]:
import os
from unicodedata import bidirectional
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import tensorflow as tf

import pandas as pd
# import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import TextVectorization # type: ignore

In [ ]:
!pip install pandas

In [ ]:
csv_file_path = "train.csv"


In [ ]:
csv_file_path = "data/train.csv"


In [ ]:
csv_file_path = "train.csv"
from google.colab import files

uploaded = files.upload()




TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
import os
filename = 'train.csv'

df = pd.read_csv(filename)

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
import tensorflow as tf

x=df['comment_text']
y=df[df.columns[2:]].values
MAX_FEATURES=200000 #number of words in the vocab

vectorizer=TextVectorization(max_tokens=MAX_FEATURES,
                                output_sequence_length=1800,
                                output_mode='int')

vectorizer.adapt(x.values) #teach the vocabulizer a vocablury
# adapt se hum learn kr lenge

# print(vectorizer('hello world'))
vectorizer.get_vocabulary()

vectorizer_text=vectorizer(x.values) # tokenize krrha hai yh

print(vectorizer_text)


# now creating a tensorflow data pipeline


#MCSHBAP -> MAP,CHACHE,SHUFFLE,BATCH,PERFETCH from_tensor_slices, list_flie
dataset=tf.data.Dataset.from_tensor_slices((vectorizer_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

batch_x,batch_y=dataset.as_numpy_iterator().next()

print(batch_x.shape)

train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

train_generator=train.as_numpy_iterator()
print(train_generator.next())


from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding # type: ignore


model=Sequential()
#create the embedding layer
model.add(Embedding(MAX_FEATURES+1,32))
#bidirectional lstm layer iska mtlb ki jaise hai i don't hate you to isme backward aur forward dekhega
model.add(Bidirectional(LSTM(32,activation='tanh')))
#feature extractor fully conn layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#final layer
model.add(Dense(6,activation='sigmoid')) # convert krte hai value to 0 to 1 mai sigmoid se paas hote hai

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
print(model.summary())

In [ ]:

from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding # type: ignore


model=Sequential()
#create the embedding layer
model.add(Embedding(MAX_FEATURES+1,32))
#bidirectional lstm layer iska mtlb ki jaise hai i don't hate you to isme backward aur forward dekhega
model.add(Bidirectional(LSTM(32,activation='tanh')))
#feature extractor fully conn layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#final layer
model.add(Dense(6,activation='sigmoid')) # convert krte hai value to 0 to 1 mai sigmoid se paas hote hai

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
print(model.summary())


history =model.fit(train,epochs=1,validation_data=val)

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# import tensorflow as tf
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
print(plt.show())


input_text=vectorizer('You freaking suck!')

batch=test.as_numpy_iterator().next()
batch_x, batch_y=test.as_numpy_iterator().next()
(model.predict(batch_x) > 0.5).astype(int)

print(input_text)

res=model.predict(batch_x)
res.flatten() #converts to one big array

 #EVALUATEING MODEL

from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy # type: ignore

pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

for batch in test.as_numpy_iterator():
    x_true, y_true=batch
    #make a prediction
    yhat=model.predict(x_true)
    #falttena prediction
    y_true=y_true.flatten()
    yhat=yhat.flatten()

    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)

print(f'Precision: {pre.result().numpy},Recall:{re.result().numpy},Accuracy:{acc.result().numpy}')



In [ ]:
!pip install -q gradio


In [ ]:
import gradio as gr

In [ ]:
input_str=vectorizer('hey i freaken hate you!')

In [ ]:
res=model.predict(np.expand_dims(input_str,0))

In [ ]:
df.columns[2:]

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment=vectorizer([comment])
    results=model.predict(vectorized_comment)

    text=''
    for idx, col in enumerate(df.columns[2:]):
        text+= '{}: {}\n'.format(col,results[0][idx]>0.5)
    return text

In [ ]:
!pip install --upgrade gradio


In [ ]:
interface = gr.Interface(
    fn=score_comment,
    inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
    outputs=gr.Textbox()
)

In [ ]:
interface.launch(share=True)